In [17]:
import pandas as pd
import os

#read a csv file and assign it to dataframe object df
df= pd.read_csv("/home/ubuntu/Desktop/output/output3.csv", header=None, names=['Column'])


#paths of benign and malcious files' folder that the yara rule was tested on
malicous_folder = '/home/ubuntu/Downloads/malwareFolder/mixedSamples/malicous'
benign_folder = '/home/ubuntu/Downloads/malwareFolder/mixedSamples/benign'

#creating dataframe for the benign files
benign_files = [f for f in os.listdir(benign_folder)]
benign_df = pd.DataFrame({'File': benign_files, 'Category': 'benign'})

#creating dataframe for the malcious files

malicous_files = [f for f in os.listdir(malicous_folder)]
malicous_df = pd.DataFrame({'File': malicous_files, 'Category': 'malicous'})

#reading the YARA detected file list from the CSV
detected_df = pd.read_csv("/home/ubuntu/Desktop/output/output3.csv", header=None, names=['Rule'])
detected_df['Category'] = detected_df['Rule'].str.split('/').str[1]
detected_df['File'] = detected_df['Rule'].str.split('/').str[2]


detected_df.head()


,Rule,Category,File
0,testRule3 ./malicous/7648612366e6978b4037c2ead...,malicous,7648612366e6978b4037c2eadfe552714ec7790f8fdaa1...
1,testRule3 ./malicous/0b1c6998a9ec410fcb9b729ca...,malicous,0b1c6998a9ec410fcb9b729ca0e811672db0e6bbfcc578...
2,testRule3 ./malicous/11668696918a1745480fa2d14...,malicous,11668696918a1745480fa2d147064b39712995877417cb...
3,testRule3 ./malicous/389407258294aaa4f1e40c92f...,malicous,389407258294aaa4f1e40c92f24935786b3fa0b2b4b6d8...
4,testRule3 ./malicous/9088306873eda32a14e383458...,malicous,9088306873eda32a14e383458448828dba3e6fa0a65654...


In [20]:
benign_df.head()

,File,Category
0,0fe87602f6f2a6dd5acf209eea81ec0555fe1e02d6f781...,benign
1,0dfd9b9733913d7e1fc1ca19798b9ed57c2d4138ae5fa1...,benign
2,0c8af4bbf3e29c4ab7dd8372725a931985b669d15b8261...,benign
3,0fbe33fb688bc8b21db405ffc08f696f5eea215264952f...,benign
4,0d23b6f46e848a65911518e6658488070a7462cd605cfc...,benign


In [19]:
malwareGroups = detected_df.groupby(['Category'])
#size of each group
malwareGroups.size()

Category
benign       1
malicous    76
dtype: int64

In [24]:
#which malicous file was not detected 
uncommon_files = malicous_df[~malicous_df["File"].isin(detected_df["File"])]["File"]
print(uncommon_files)

30    0acf541f983ebcbc34a65128f894228237d3505ab231db...
Name: File, dtype: object


In [38]:
#calculating TP (True Positive)
TP = sum(detected_df["File"].isin(malicous_df["File"]) & (detected_df['Category'] == 'malicous'))

#calculating FP (False Positive)
FP = sum(detected_df["File"].isin(benign_df["File"]) & (detected_df['Category'] == 'benign'))

#calculating TN (True Negative) - everything in benign folder that wasn't detected by YARA
TN = benign_df[~benign_df["File"].isin(detected_df["File"])].shape[0]

#calculating FN (False Negative) - everything in malicous folder that wasn't detected by YARA
FN = malicous_df[~malicous_df["File"].isin(detected_df["File"])].shape[0]


#Calculating metrics
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1_Score = 2 * (Precision * Recall) /(Precision + Recall)
Accuracy = (TP + TN) / (TP + FP + TN + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)


print("Precison:", Precision )
print("Recall:", Recall )
print("F1_Score:", F1_Score )
print("Accuracy:", Accuracy )
print("TPR:", TPR )
print("FPR:", FPR )



Precison: 0.987012987012987
Recall: 0.987012987012987
F1_Score: 0.987012987012987
Accuracy: 0.9809523809523809
TPR: 0.987012987012987
FPR: 0.03571428571428571
